In [1]:
import pandas as pd
import numpy as np

In [10]:
df = pd.read_excel('lotto.xlsx',header=2)
df = df.iloc[:,-7:-1]
df.head()

,1,2,3,4,5,6
0,14,19,27,28,30,45
1,22,26,29,30,34,45
2,5,17,26,27,35,38
3,21,26,30,32,33,35
4,6,12,31,35,38,43


In [24]:
# 수집한 로또 데이터는 최신 데이터가 앞에 있으므로.. 과거순으로 reverse 한다.
reverse_index = np.array(range(1054))[::-1]
lotto_df = df.iloc[reverse_index,:]
lotto_df = lotto_df.reset_index(drop=True)
lotto_df.head()

,1,2,3,4,5,6
0,10,23,29,33,37,40
1,9,13,21,25,32,42
2,11,16,19,21,27,31
3,14,27,30,31,40,42
4,16,24,29,40,41,42


In [35]:
# d = {'1': [4], '2': [7], '3':[12],'4':[14],'5':[22],'6':[33]}
# temp = pd.DataFrame(data=d)
# pd.concat([lotto_df,temp],axis=0)


In [41]:
lotto_df.loc[1054] = [4,7,12,14,22,33]

In [42]:
lotto_df.tail()

,1,2,3,4,5,6
1050,21,26,30,32,33,35
1051,5,17,26,27,35,38
1052,22,26,29,30,34,45
1053,14,19,27,28,30,45
1054,4,7,12,14,22,33


In [48]:
# window side = 5
windows = 5
lotto_1 = lotto_df.loc[:,1].to_numpy()
print(lotto_1[ : windows] , lotto_1[windows])
print(lotto_1[ 1: windows+1] , lotto_1[windows+1])

[10  9 11 14 16] 14
[ 9 11 14 16 14] 2


In [50]:
lotto_1_train = []
lotto_1_target = []
for i in range(len(lotto_df)):
    if windows+i <= len(lotto_df)-1:
        lotto_1_train.append( lotto_1[ i: windows+i] )
        lotto_1_target.append(lotto_1[windows+i])

In [65]:
lotto_df.to_csv("current_lotto.csv",index=0)

In [66]:
def makeNumbers(number):
    lotto_train = []
    lotto_target = []
    lotto = lotto_df.loc[:,number].to_numpy()
    for i in range(len(lotto_df)):
        if windows+i <= len(lotto_df)-1:
            lotto_train.append( lotto[ i: windows+i] )
            lotto_target.append(lotto[windows+i])
    return lotto_train,lotto_target

In [67]:
lotto_df = pd.read_csv("current_lotto.csv")

In [105]:
# 모델 적용
from xgboost import XGBRegressor
xgbr = XGBRegressor()
train,target = makeNumbers('1')
xgbr.fit(train,target)
predict_data =  lotto_df.tail().loc[:,'1'].to_numpy()
np.round(xgbr.predict([predict_data]))

array([1.], dtype=float32)

In [106]:
from xgboost import XGBRegressor
xgbr = XGBRegressor()
train,target = makeNumbers('2')
xgbr.fit(train,target)
predict_data =  lotto_df.tail().loc[:,'2'].to_numpy()
np.round(xgbr.predict([predict_data]))

array([12.], dtype=float32)

In [124]:

lotto_df.to_csv("current_lotto.csv",index=0)

for i in range(1,6):
    windows=i*2
    def makeNumbers(number):
        lotto_train = []
        lotto_target = []
        lotto = lotto_df.loc[:,number].to_numpy()
        for i in range(len(lotto_df)):
            if windows+i <= len(lotto_df)-1:
                lotto_train.append( lotto[ i: windows+i] )
                lotto_target.append(lotto[windows+i])
        return lotto_train,lotto_target
    result = []
    for i in ['1','2','3','4','5','6']:
        from xgboost import XGBRegressor
        xgbr = XGBRegressor(random_state=0)
        train,target = makeNumbers(i)
        xgbr.fit(train,target)
        predict_data =  lotto_df.tail(windows).loc[:,i].to_numpy()
        print(np.round(xgbr.predict([predict_data])))
    print("=====================================================================================")

[6.]
[10.]
[27.]
[29.]
[22.]
[39.]
[1.]
[14.]
[20.]
[23.]
[29.]
[40.]
[1.]
[17.]
[16.]
[22.]
[30.]
[37.]
[4.]
[13.]
[16.]
[26.]
[35.]
[38.]
[6.]
[13.]
[14.]
[24.]
[31.]
[39.]


In [114]:
lotto_df.tail(10)

,1,2,3,4,5,6
1045,7,16,25,29,35,36
1046,2,20,33,40,42,44
1047,6,12,17,21,32,39
1048,3,5,13,20,21,37
1049,6,12,31,35,38,43
1050,21,26,30,32,33,35
1051,5,17,26,27,35,38
1052,22,26,29,30,34,45
1053,14,19,27,28,30,45
1054,4,7,12,14,22,33
